In [1]:
import tensorflow as tf
import pandas as pd

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

# 加载数据
def load_data(y_name='Species'):
    train_path = '../data/iris_training.csv'
    test_path = '../data/iris_test.csv'
    train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
    train_x, train_y = train, train.pop(y_name)
    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y = test, test.pop(y_name)
    
    return (train_x, train_y), (test_x, test_y)


# 构建feature columns
(train_x, train_y), (test_x, test_y) = load_data()
my_feature_columns = []
for key in train_x.keys():
        my_feature_columns.append(tf.feature_column.numeric_column(key=key))
        
# input_fn
def train_input_fn(features,labels,batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset

e:\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
my_checkpointing_config = tf.estimator.RunConfig(
    save_checkpoints_secs = 1,  # Save checkpoints every 20 minutes.
    keep_checkpoint_max = 10,       # Retain the 10 most recent checkpoints.
)

classifier = tf.estimator.DNNClassifier(
        feature_columns=my_feature_columns,
        # Two hidden layers of 10 nodes each.
        hidden_units=[8, 10],
        # The model must choose between 3 classes.
        n_classes=3,
        model_dir = './logs',
        config=my_checkpointing_config)


INFO:tensorflow:Using config: {'_evaluation_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000356629F9B0>, '_service': None, '_session_config': None, '_task_id': 0, '_log_step_count_steps': 100, '_master': '', '_keep_checkpoint_max': 10, '_save_checkpoints_steps': None, '_tf_random_seed': None, '_model_dir': './logs', '_task_type': 'worker', '_save_checkpoints_secs': 1, '_num_worker_replicas': 1, '_is_chief': True, '_save_summary_steps': 100, '_global_id_in_cluster': 0, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000}


In [3]:
classifier.train(
    input_fn=lambda:train_input_fn(train_x, train_y, 100),
    steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./logs\model.ckpt.
INFO:tensorflow:step = 0, loss = 164.56326
INFO:tensorflow:global_step/sec: 267.734
INFO:tensorflow:step = 100, loss = 25.176874 (0.376 sec)
INFO:tensorflow:global_step/sec: 386.344
INFO:tensorflow:step = 200, loss = 12.988299 (0.259 sec)
INFO:tensorflow:Saving checkpoints for 213 into ./logs\model.ckpt.
INFO:tensorflow:global_step/sec: 173.115
INFO:tensorflow:step = 300, loss = 8.41159 (0.578 sec)
INFO:tensorflow:global_step/sec: 386.664
INFO:tensorflow:step = 400, loss = 8.126438 (0.259 sec)
INFO:tensorflow:Saving checkpoints for 496 into ./logs\model.ckpt.
INFO:tensorflow:global_step/sec: 129.119
INFO:tensorflow:step = 500, loss = 7.6150527 (0.773 sec)
INFO:tensorflow:global_step/sec: 5

In [4]:
eval_result = classifier.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y, 100))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-14-14:11:40
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./logs\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-14-14:11:41
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.96666664, average_loss = 0.053858127, global_step = 1000, loss = 1.6157439


In [6]:
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

predictions = classifier.predict(
    input_fn=lambda:eval_input_fn(predict_x, labels=None, batch_size=100))

In [8]:
list(predictions)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./logs\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object),
  'logits': array([ 14.055865,   8.990452, -17.106209], dtype=float32),
  'probabilities': array([9.937283e-01, 6.271718e-03, 2.909045e-14], dtype=float32)},
 {'class_ids': array([1], dtype=int64),
  'classes': array([b'1'], dtype=object),
  'logits': array([-1.6522006 ,  5.7732453 , -0.15836635], dtype=float32),
  'probabilities': array([5.9396459e-04, 9.9676043e-01, 2.6456027e-03], dtype=float32)},
 {'class_ids': array([2], dtype=int64),
  'classes': array([b'2'], dtype=object),
  'logits': array([-7.899126 ,  4.509861 ,  7.5360513], dtype=float32),
  'probabilities': array([1.8880753e-07, 4.6256598e-02, 9.5374316e-01], dtype=float32)}]

In [16]:
import numpy as np
B = np.arange(-10,10).reshape(-1,5).astype(np.float32)
B

array([[-10.,  -9.,  -8.,  -7.,  -6.],
       [ -5.,  -4.,  -3.,  -2.,  -1.],
       [  0.,   1.,   2.,   3.,   4.],
       [  5.,   6.,   7.,   8.,   9.]], dtype=float32)

In [17]:
import tensorflow as tf
with tf.Session() as sess:
    A = sess.run(tf.nn.softmax(B))
    print(A)

[[0.01165623 0.03168492 0.08612854 0.23412165 0.6364086 ]
 [0.01165623 0.03168492 0.08612854 0.23412165 0.6364086 ]
 [0.01165623 0.03168492 0.08612854 0.23412165 0.6364086 ]
 [0.01165623 0.03168492 0.08612854 0.23412165 0.6364086 ]]


In [77]:
for pred_dict, expec in zip(predictions, expected):
    template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print(template.format(SPECIES[class_id], 100 * probability, expec))
    

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./logs\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "Setosa" (99.7%), expected "Setosa"

Prediction is "Versicolor" (99.6%), expected "Versicolor"

Prediction is "Virginica" (98.2%), expected "Virginica"
